In [41]:
#!/usr/bin/env python3
"""
compute_waveforms_and_metrics.py

Given your core_dir and list of session names, this will:
  1. Load the behavioral (and, if present, tagging) sorting_analyzers
  2. For each unit, extract median & mean waveforms via your waveformPrepare_template()
  3. Compute template_metrics, correlograms & isi_histograms
  4. Save everything as .npy under each session’s kilosort4 folder
"""

import os
import sys
import pathlib
sys.path.append(str(pathlib.Path.cwd() / "pipeline" / "spikeinterface_waveform_extraction"))

import numpy as np
from spikeinterface_v2 import load_sorting_analyzer
from my_spike_tools import waveformPrepare_template  # adjust import as needed

# 1) Configuration — point these at your setup
core_dir = r"/Volumes/Extreme SSD/Neuropixels"
date_strs = [
    '01292025', '01302025', '01312025', '02012025',
    '02022025', '02032025', '02042025', '02062025', '02072025',
    '02092025', '02102025', '02112025', '02122025', '02132025',
    '02142025', '02152025', '02162025'
]
session_names = [f"9153_{d}_tagging_g0" for d in date_strs]

for session in session_names:
    sel = os.path.join(core_dir, session, f"{session}_imec0")
    kilo_dir = os.path.join(sel, "kilosort4")

    # 2) load analyzers
    beh_folder = os.path.join(sel, "analyzer_beh")
    beh_an = load_sorting_analyzer(folder=beh_folder, format="binary_folder")

    tag_folder = os.path.join(sel, "analyzer_tag")
    has_tag = os.path.isdir(tag_folder)
    if has_tag:
        tag_an = load_sorting_analyzer(folder=tag_folder, format="binary_folder")

    # get all unit IDs
    unit_ids = beh_an.sorting.unit_ids

    # prepare containers
    wf_beh_med, wf_beh_avg = {}, {}
    wf_tag_med, wf_tag_avg = {}, {}

    # 3) extract waveforms
    for uid in unit_ids:
        avg, med, _ = waveformPrepare_template(
            beh_an, unit_id=uid, designated=False, use_template=True
        )
        wf_beh_avg[uid] = avg
        wf_beh_med[uid] = med

        if has_tag:
            avg_t, med_t, _ = waveformPrepare_template(
                tag_an, unit_id=uid, designated=True, use_template=False
            )
            wf_tag_avg[uid] = avg_t
            wf_tag_med[uid] = med_t

    # compute and save metrics
    for ext in ("template_metrics","correlograms","isi_histograms"):
        os.makedirs(os.path.join(beh_folder, "extensions", ext), exist_ok=True)
    tm  = beh_an.compute(
        input="template_metrics", include_multi_channel_metrics=True, save=False
    )
    ccg = beh_an.compute(
        input="correlograms", window_ms=1000.0, bin_ms=5.0, method="auto", save=False
    )
    isi = beh_an.compute(
        input="isi_histograms", window_ms=1000.0, bin_ms=5.0, method="auto", save=False
    )

    # grab your unit IDs
    unit_ids = beh_an.sorting.unit_ids

    # convert metrics → dicts by unit_id
    # Convert template metrics to a dictionary manually
    ccg_dict = { uid: ccg.get_data()[0][i, i]  for i, uid in enumerate(unit_ids) }
    isi_dict = { uid: isi.get_data()[0][i]     for i, uid in enumerate(unit_ids) }

    # save metrics
    tm.get_data().to_csv(os.path.join(kilo_dir, "template_metrics.csv"))
    np.save(os.path.join(kilo_dir, "ACC.npy"),      ccg_dict)
    np.save(os.path.join(kilo_dir, "ISI.npy"),    isi_dict)

    # save waveforms
    np.save(os.path.join(kilo_dir, "waveform_beh_average.npy"), wf_beh_avg)
    np.save(os.path.join(kilo_dir, "waveform_beh_median.npy"),  wf_beh_med)
    if has_tag:
        np.save(os.path.join(kilo_dir, "waveform_tag_average.npy"), wf_tag_avg)
        np.save(os.path.join(kilo_dir, "waveform_tag_median.npy"),  wf_tag_med)

    print(f"[{session}] done.")

Number of spikes for unit 3: 657
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 3: 6
Calculating average and median from individual spikes for unit 3...
Number of spikes for unit 5: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 5: 82
Calculating average and median from individual spikes for unit 5...
Number of spikes for unit 6: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 6: 520
Calculating average and median from individual spikes for unit 6...
Number of spikes for unit 7: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 7: 30
Calculating average and median from individual spikes for unit 7...
Number of spikes for unit 9: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for 

/opt/anaconda3/lib/python3.12/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/opt/anaconda3/lib/python3.12/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/opt/anaconda3/lib/python3.12/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/

[9153_01292025_tagging_g0] done.
Number of spikes for unit 0: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 0: 3
Calculating average and median from individual spikes for unit 0...
Number of spikes for unit 2: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 2: 277
Calculating average and median from individual spikes for unit 2...
Number of spikes for unit 3: 54
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 3: 0
Number of spikes for unit 5: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 5: 53
Calculating average and median from individual spikes for unit 5...
Number of spikes for unit 6: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 6: 43
Calculating average and me

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regresso

[9153_01302025_tagging_g0] done.
Number of spikes for unit 1: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 1: 38
Calculating average and median from individual spikes for unit 1...
Number of spikes for unit 3: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 3: 350
Calculating average and median from individual spikes for unit 3...
Number of spikes for unit 4: 684
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 4: 31
Calculating average and median from individual spikes for unit 4...
Number of spikes for unit 5: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 5: 43
Calculating average and median from individual spikes for unit 5...
Number of spikes for unit 6: 1000
Using designated_templates for potential fallback template computatio

...

[9153_02142025_tagging_g0] done.
Number of spikes for unit 1: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 1: 48
Calculating average and median from individual spikes for unit 1...
Number of spikes for unit 2: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 2: 971
Calculating average and median from individual spikes for unit 2...
Number of spikes for unit 5: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 5: 23
Calculating average and median from individual spikes for unit 5...
Number of spikes for unit 6: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 6: 87
Calculating average and median from individual spikes for unit 6...
Number of spikes for unit 7: 1000
Using designated_templates for potential fallback template computati

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regresso

[9153_02152025_tagging_g0] done.
Number of spikes for unit 0: 887
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 0: 22
Calculating average and median from individual spikes for unit 0...
Number of spikes for unit 4: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 4: 199
Calculating average and median from individual spikes for unit 4...
Number of spikes for unit 5: 395
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 5: 5
Calculating average and median from individual spikes for unit 5...
Number of spikes for unit 10: 1000
Using designated_templates for potential fallback template computation if needed.
Number of spikes for unit 10: 173
Calculating average and median from individual spikes for unit 10...
Number of spikes for unit 11: 1000
Using designated_templates for potential fallback template computa

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_theil_sen.py:128: ConvergenceWarning: Maximum number of iterations 300 reached in spatial median for TheilSen regressor.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning: invalid value encountered in scalar divide
  slope = ssxym / ssxm
/opt/anaconda3/lib/python3.12/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
/opt/anaconda3/lib/python3.12/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning: invalid value encountered in scalar divide
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/opt/anaconda3/lib/python3.12/

[9153_02162025_tagging_g0] done.
